In [28]:
import polars as pl
import statsmodels.formula.api as smf
import numpy as np
from stargazer.stargazer import Stargazer
from linearmodels.panel import PanelOLS

In [29]:
countries = set(open("./worldbankcountries.txt").read().split())
df = pl.read_parquet("../data/combined-2022-xgboost-synthetic-eiu.parquet")
df = df.filter(pl.col("country").str.contains_any(list(countries)))
pd_df = df.to_pandas()
pd_df = pd_df[pd_df.reset_index()["year"].isin([2003, 2006, 2009, 2012, 2015, 2018, 2022])]

pd_df = pd_df.set_index(["country", "year"])

In [31]:
# pd_df["arwu_ranked_num_pc"] = pd_df["arwu_ranked_num"] / pd_df["population"] * 1_000_000
# pd_df["gdp_pc"] = np.log(pd_df["gdp_pc"])
# pd_df["population"] = np.log(pd_df["population"])

In [32]:
group1 = pd_df[(pd_df["arwu_ranked_num"].notnull()) & (pd_df["math"].notnull()) & (pd_df["imo_total_score"].notnull()) & (pd_df["primary_completion"].notnull()) & (pd_df["lower_sec_completion"].notnull()) & (pd_df["upper_sec_completion"].notnull()) & (pd_df["democracy_eiu"].notnull())]
group1.shape

(441, 17)

In [33]:
group2 = pd_df[(pd_df["gdp_pc"].notnull()) & (pd_df["arwu_ranked_num"].notnull()) & (pd_df["imo_total_score"].notnull()) & (pd_df["primary_completion"].notnull()) & (pd_df["lower_sec_completion"].notnull()) & (pd_df["upper_sec_completion"].notnull()) & (pd_df["democracy_eiu"].notnull())]
group2.shape

(1446, 17)

In [34]:
pisa_time_fe = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + TimeEffects", group1).fit()

/Users/jamesxu/miniforge3/envs/442research/lib/python3.12/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [35]:
pisa_time_country_fe = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + TimeEffects + EntityEffects", group1).fit()

/Users/jamesxu/miniforge3/envs/442research/lib/python3.12/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [36]:
non_pisa_pyears = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + TimeEffects", pd_df).fit()

/Users/jamesxu/miniforge3/envs/442research/lib/python3.12/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [37]:
non_pisa_pyears

Dep. Variable:,gdp_pc_growth,R-squared:,0.0208
Estimator:,PanelOLS,R-squared (Between):,0.2144
No. Observations:,1424,R-squared (Within):,-0.0045
Date:,"Tue, Apr 23 2024",R-squared (Overall):,0.0582
Time:,19:07:43,Log-likelihood,-1.105e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.2847
Entities:,213,P-value,0.0001
Avg Obs:,6.6854,Distribution:,"F(7,1410)"
Min Obs:,1.0000,,
Max Obs:,7.0000,F-statistic (robust):,4.2847


In [8]:
pisa_no_math_time_fe = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + imo_total_score + in_math99 + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + TimeEffects", group1).fit()

/Users/jamesxu/miniforge3/envs/442research/lib/python3.12/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [9]:
non_pisa = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + TimeEffects", group2).fit()

/Users/jamesxu/miniforge3/envs/442research/lib/python3.12/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [10]:
non_pisa_no_interac = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + TimeEffects", group2).fit()

/Users/jamesxu/miniforge3/envs/442research/lib/python3.12/site-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [11]:
pisa_time_country_fe

Dep. Variable:,gdp_pc_growth,R-squared:,0.0767
Estimator:,PanelOLS,R-squared (Between):,0.2030
No. Observations:,440,R-squared (Within):,0.0416
Date:,"Sat, Apr 20 2024",R-squared (Overall):,0.0408
Time:,12:36:28,Log-likelihood,-3132.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.5236
Entities:,89,P-value,0.0046
Avg Obs:,4.9438,Distribution:,"F(11,334)"
Min Obs:,1.0000,,
Max Obs:,7.0000,F-statistic (robust):,2.5236
